In [1]:
import os

os.environ['DJANGO_SETTINGS_MODULE'] = 'cashup_backend.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

import django
django.setup()

In [2]:
from data.models import HourData

In [3]:
import requests
from datetime import timedelta, datetime

In [4]:
up_dn_list = []
last_data = []
dn_list = []
up_list = []
new_dn_list = []
new_up_list = []
other_up_dn_list = []

continue_up_down = "D0"
prev_up_down = "S"
work_1_0_1 = "S"
work_1_5_1 = "S"
work_1_0_2 = "S"
work_1_5_2 = "S"
last_time_1_0_1 = datetime.now()
last_time_1_5_1 = datetime.now()
last_time_1_0_2 = datetime.now()
last_time_1_5_2 = datetime.now()

last_dn, last_up = "", ""

max_price, min_price = 0, 0
work_up_high, work_dn_low = 1000000, 0
volume = 0
prev_low = 0
prev_high = 100000
start = 0
last_up = 0
last_dn = 0
volume_up_dn = ""
hour_up_down = ""
prev_signal_up_price = 0
prev_signal_down_price = 0

In [ ]:

while True:
    URL = "https://www.bitmex.com/api/v1/trade/bucketed?symbol=XBT&binSize=1h&partial=false&start={}&count=1000&reverse=false".format(start)
    req = requests.get(URL).json()
    for data in req:
        volume_up_dn = ""
        signal = ""
        signal_price = 0
        hour_up_down = ""
        now_work_1_0_1 = ""
        now_work_1_5_1 = ""
        now_work_1_0_2 = ""
        now_work_1_5_2 = ""
        datetime = datetime.strptime(data['timestamp'].replace("T", " ")[0:19], "%Y-%m-%d %H:%M:%S") + timedelta(hours=8)
        open_price = data['open'] if data['open'] != None else 0
        high_price = data['high'] if data['high'] != None else 0
        low_price = data['low'] if data['low'] != None else 0
        close_price = data['close'] if data['close'] != None else 0
        avg_price = (high_price + low_price) / 2
        
        if volume != 0:
            volume_rate = data['volume'] / volume
        else:
            volume_rate = 0
            
        volume = data['volume']

        flag = False

        if open_price > close_price:
            UD = "D"

        elif open_price < close_price:
            UD = "U"

        else:
            UD = "S"

        if len(last_data) > 3:
            last_data.pop(0)
            volume_bigger_count = 0
            volume_smaller_count = 0

            for i in last_data:
                if i[2] < volume:
                    volume_bigger_count += 1

                if i[2] > volume:
                    volume_smaller_count += 1

            if volume_bigger_count == 3:
                volume_up_dn = "UP"

            if volume_smaller_count == 3:
                volume_up_dn = "DN"

        last_data.append([low_price, high_price, volume])
        
        if continue_up_down[0] == "U":
            if UD == "U":
                continue_up_down = "U" + str(int(continue_up_down[1:]) + 1)
            elif UD == "D":
                continue_up_down = "D1"
        else:
            if UD == "U":
                continue_up_down = "U1"
            elif UD == "D":
                continue_up_down = "D" + str(int(continue_up_down[1:]) + 1)
        
        if len(up_dn_list) > 3:
            up_dn_list.pop(0)
        up_dn_list.append(UD)
        
        if up_dn_list.count("U") > 2:
            if high_price >= last_up:
                hour_up_down = "UP(U)"
            else:
                hour_up_down = "UP(D)"
            last_up = high_price
        
        if up_dn_list.count("D") > 2:
            if low_price >= last_dn:
                hour_up_down = "DN(U)"
            else:
                hour_up_down = "DN(D)"
            last_dn = low_price
            
        if UD == "U":
            if other_up_dn_list.count("D") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                for idx, element in enumerate(other_up_dn_list):
                    if element == "D":
                        count += 1
                        max_price_list.append(dn_list[idx])
                    elif element == "U":
                        if count >= 3:
                            break
                    min_price_list.append(up_list[idx])

                if prev_up_down == "D" and max(max_price_list) == max_price_list[len(max_price_list) - 1]:
                    if prev_signal_down_price != 0:
                        if prev_signal_down_price < min(min_price_list):
                            signal = "fD(U)"
                        elif prev_signal_down_price == min(min_price_list):
                            signal = "fD(0)"
                        else:
                            signal = "fD(D)"
                    prev_signal_down_price = min(min_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()

        if UD == "D":
            if other_up_dn_list.count("U") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                is_D = False
                for idx, element in enumerate(other_up_dn_list):
                    if element == "U":
                        count += 1
                        min_price_list.append(up_list[idx])
                    elif element == "D":
                        if count >= 3:
                            break
                    max_price_list.append(dn_list[idx])

                if prev_up_down == "U" and min(min_price_list) == min_price_list[len(min_price_list) - 1]:
                    print(datetime)
                    print(other_up_dn_list)
                    print(max_price_list, min_price_list)
                    if prev_signal_down_price != 0:
                        if prev_signal_up_price < max(max_price_list):
                            signal = "fU(U)"
                        elif prev_signal_up_price == max(max_price_list):
                            signal = "fU(0)"
                        else:
                            signal = "fU(D)"
                    prev_signal_up_price = max(max_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
        
        if len(dn_list) > 5:
            dn_list.pop(0)
        dn_list.append(high_price)
            
        if len(up_list) > 5:
            up_list.pop(0)
        up_list.append(low_price)
        
        if len(other_up_dn_list) > 5:
            other_up_dn_list.pop(0)
        other_up_dn_list.append(UD)
        
        if volume_up_dn == "UP":
            if (max_price < high_price) and (volume_rate > 1):
                if work_1_0_1[0] == "U":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0_1 = True
                    else:
                        work_1_0_1 = "U" + str(int(work_1_0_1[1:]) + 1)
                else:
                    work_1_0_1 = "U1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1):
                if work_1_0_1[0] == "D":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0 = True
                    else:
                        work_1_0_1 = "D" + str(int(work_1_0_1[1:]) + 1)
                
                else:
                    work_1_0_1 = "D1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
                
            if (max_price < high_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "U":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        continue_flag_1_5_1 = True
                    else:
                        work_1_5_1 = "U" + str(int(work_1_5_1[1:]) + 1)
                else:
                    work_1_5_1 = "U1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_1 = "D" + str(int(work_1_5_1[1:]) + 1)
                
                else:
                    work_1_5_1 = "D1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime

            if (max_price < high_price and min_price < low_price) and (volume_rate > 1):
                if work_1_0_2[0] == "U":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "U" + str(int(work_1_0_2[1:]) + 1)
                else:
                    work_1_0_2 = "U1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1):
                if work_1_0_2[0] == "D":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "D" + str(int(work_1_0_2[1:]) + 1)
                
                else:
                    work_1_0_2 = "D1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime     
                
                
            if (max_price < high_price and min_price < low_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "U":
                    if datetime == last_time_1_5_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "U" + str(int(work_1_5_2[1:]) + 1)
                else:
                    work_1_5_2 = "U1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "D" + str(int(work_1_5_2[1:]) + 1)
                
                else:
                    work_1_5_2 = "D1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime

            max_price, min_price = high_price, low_price
            
        if now_work_1_0_1 != "":
            now_work_1_0_1 = "w" + now_work_1_0_1
        
        if now_work_1_5_1 != "":
            now_work_1_5_1 = "w" + now_work_1_5_1
            
        if now_work_1_0_2 != "":
            now_work_1_0_2 = "w" + now_work_1_0_2
            
        if now_work_1_5_2 != "":
            now_work_1_5_2 = "w" + now_work_1_5_2
        
        print(hour_up_down)
        element = HourData()
        element.datetime = datetime
        element.open_price = open_price
        element.min_price = low_price
        element.max_price = high_price
        element.close_price = close_price
        element.avg_price = avg_price
        element.volume = volume
        element.volume_rate = volume_rate
        element.volume_up_dn = volume_up_dn
        element.up_down = UD
        element.continue_up_down = continue_up_down
        element.hour_up_down = hour_up_down
        element.signal = signal
        element.signal_price = signal_price
        element.work_two_1_0 = now_work_1_0_1
        element.work_two_1_5 = now_work_1_5_1
        element.work_one_1_0 = now_work_1_0_2
        element.work_one_1_5 = now_work_1_5_2
        element.save()
        
        prev_up_down = UD
        
    start += 1000




DN(U)




DN(D)
DN(D)
DN(U)

UP(U)
2015-09-26 10:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[235.29, 235.36, 235.2] [235.29, 235.19, 234.9]
UP(U)








DN(D)
DN(D)


UP(D)
2015-09-27 00:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[235, 234.35, 234.87, 233.85] [235, 234.1, 232.91]

DN(U)


UP(D)
UP(D)
2015-09-27 06:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[234.67, 234.73, 234, 233.8] [234.32, 234.05, 233.69, 233.68]
UP(D)
UP(D)












DN(D)
DN(U)
DN(D)





UP(D)


DN(U)
DN(D)
DN(D)
DN(U)

UP(U)
UP(D)
UP(D)
2015-09-28 15:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[237.8, 237.82, 238.23, 234.58, 233.93] [237.37, 237.34, 234.45, 233.6, 233.39]
UP(D)
UP(U)
2015-09-28 17:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[238.7, 237.24, 237.8, 237.82, 238.23, 234.58] [237.3, 237.37, 237.34, 234.45, 233.6]


UP(U)
2015-09-28 21:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[239.16, 238.6, 240.49, 238.7] [238.57, 238.31, 237.3]

UP(D)
UP(U)
UP(D)
2015-09-29 01:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[239.31, 239.7, 238.19] [238.8,


2015-10-17 05:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[266.02, 265.08, 267.55, 266.22, 265.59, 262.87] [265, 264, 262.92, 261.85]
DN(U)



UP(D)
2015-10-17 10:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[264.52, 263.89, 261.84, 262.82] [263, 261.84, 261.1]

UP(U)
2015-10-17 12:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[266.95, 264.75, 264.52, 263.89] [264.41, 263, 261.84]


2015-10-17 14:00:00
['U', 'D', 'U', 'D', 'U', 'U']
[270.39, 267.55, 266.95, 264.75, 264.52, 263.89] [267.19, 264.41, 263, 261.84]

DN(U)


UP(U)
UP(D)
UP(U)
2015-10-17 21:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[272.09, 271.1, 271.79, 270.74, 268.67] [270.78, 269.62, 270.01, 268.2, 266.2]
UP(U)
UP(D)
2015-10-17 23:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[275, 278.51, 272.09, 271.1, 271.79, 270.74] [270.5, 270.78, 269.62, 270.01, 268.2]


2015-10-18 01:00:00
['U', 'D', 'U', 'D', 'U', 'U']
[274.71, 274.7, 275, 278.51, 272.09, 271.1] [271.75, 270.5, 270.78, 269.62]


2015-10-18 03:00:00
['U', 'D', 'U', 'D', 'U', 'D']
[273.77, 274.8, 274.7

DN(D)
DN(U)
DN(D)
DN(D)
DN(D)
DN(D)
DN(D)
DN(D)
DN(U)
DN(D)



UP(D)
2015-11-01 10:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[321.29, 317.8, 319.35, 317.4] [317.76, 314.48, 310.92]




UP(U)
2015-11-01 15:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[322.51, 320.71, 316.93] [317.64, 316.43, 316.33]
UP(D)
UP(D)
UP(U)
UP(U)
2015-11-01 19:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[324.44, 320.42, 319.57] [320.45, 319, 318.39]
UP(U)


2015-11-01 22:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[325.66, 323.6, 324.62, 324.44, 320.42, 319.57] [323.5, 320.45, 319, 318.39]
DN(U)
DN(D)
DN(D)
DN(D)
DN(D)

UP(U)
UP(U)
UP(U)
UP(U)
2015-11-02 08:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[334.51, 330.64, 328.27, 325.95, 321.19] [330.33, 326.22, 325.45, 320.69, 315.79]
UP(U)
UP(D)
2015-11-02 10:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[334.12, 335, 334.51, 330.64, 328.27, 325.95] [331.27, 330.33, 326.22, 325.45, 320.69]


UP(D)

UP(U)
2015-11-02 15:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[333.7, 333.53, 333.02, 332.41] [331.45, 329.06, 32

DN(U)

DN(D)
DN(U)


2015-11-17 00:00:00
['U', 'U', 'D', 'D', 'U', 'D']
[322.18, 320.34, 319.57, 320.06, 319.36] [319.19, 319.27, 318.9]

UP(U)
UP(U)


DN(U)
DN(D)
2015-11-17 07:00:00
['U', 'D', 'D', 'D', 'U', 'U']
[334.77, 331.89, 332.99, 333.76, 334, 322] [329.58, 321.32, 320.04]
DN(U)

UP(D)
UP(U)
2015-11-17 11:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[340.98, 332.86, 332.36] [331.89, 331.15, 329.97]
UP(U)

DN(U)
DN(D)
DN(U)
DN(D)
DN(D)



UP(D)

DN(U)



UP(D)
2015-11-18 04:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[337.96, 336.77, 337.5, 338.29] [336.73, 335.64, 334.5]

DN(U)





DN(D)

UP(D)









2015-11-18 23:00:00
['U', 'U', 'D', 'D', 'U', 'S']
[337.47, 334.35, 333.3, 333.4, 333.04, 333.17] [333.49, 332.9, 330.22]



2015-11-19 02:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[335.63, 336.36, 337.74, 337.47, 334.35] [335.11, 333.49, 332.9]
DN(D)
DN(U)


UP(U)
UP(D)

DN(U)
DN(D)
DN(U)




DN(D)

UP(D)
UP(D)
UP(U)
2015-11-19 21:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[334.8, 334.51, 335.05, 333.

DN(D)
DN(U)



DN(D)
DN(U)






UP(D)
2015-12-04 20:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[360.32, 360.56, 360.31, 357.4] [359.9, 357.87, 353.21]


2015-12-04 22:00:00
['U', 'D', 'U', 'D', 'U', 'U']
[360.49, 360.06, 360.32, 360.56, 360.31, 357.4] [359, 359.9, 357.87, 353.21]




2015-12-05 02:00:00
['U', 'D', 'U', 'D', 'U', 'D']
[363.22, 365.15, 364.5, 360.96, 360.49] [362.71, 359.45, 359]


UP(U)

UP(D)


UP(U)
UP(U)
2015-12-05 11:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[369.99, 365.55, 364.5] [364.5, 363.35, 362.86]
UP(U)
UP(U)
UP(D)
2015-12-05 14:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[374.45, 375.59, 371.22, 369.99, 365.55, 364.5] [371.21, 368.9, 364.5, 363.35, 362.86]

UP(D)
2015-12-05 16:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[373.5, 373.41, 374.45, 375.59] [372.28, 371.21, 368.9]

DN(U)


UP(U)
2015-12-05 21:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[381.43, 378.44, 372.44] [374.42, 372.44, 370.64]
UP(U)



UP(D)
2015-12-06 02:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[381.89, 380.69, 380.78] [3

UP(U)
UP(U)
2015-12-21 21:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[444.88, 443, 436.01, 437.17] [441.62, 436.35, 434.43, 434]
UP(U)
UP(D)
2015-12-21 23:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[443.34, 446, 444.88, 443, 436.01, 437.17] [440.5, 441.62, 436.35, 434.43, 434]

DN(U)



UP(D)
2015-12-22 05:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[440.22, 440.41, 442.17, 440.36] [434.09, 440.06, 433.05]

DN(U)


UP(U)
UP(U)
2015-12-22 11:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[443.06, 441, 439.78, 437.54] [440.4, 437.91, 436.5, 435.05]
UP(D)
UP(D)
UP(U)
2015-12-22 14:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[440.49, 440.01, 442, 443.06, 441, 439.78] [439.16, 438.46, 440.4, 437.91, 436.5]

UP(D)
UP(U)
UP(U)
UP(U)
2015-12-22 19:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[443.39, 442.39, 441.43, 439.49] [441.91, 439.75, 438.63, 438.58]
UP(D)
UP(D)
UP(U)
2015-12-22 22:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[441.28, 440.5, 442.99, 443.39, 442.39, 441.43] [439.48, 439.43, 441.91, 439.75, 438.63]







2015-12-23 05:00:0

DN(U)
DN(U)
DN(U)
DN(D)

DN(U)
DN(D)
DN(D)
DN(U)
DN(D)
DN(D)

DN(D)
DN(D)


UP(D)
2016-01-06 01:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[433.69, 432.21, 432.13] [432.2, 431.12, 429.67]
UP(U)




UP(U)
UP(D)

DN(U)
DN(U)
DN(D)
DN(D)


UP(D)
2016-01-06 16:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[431.52, 430.18, 428.78] [429.48, 428.5, 427.2]
UP(U)
UP(D)
2016-01-06 18:00:00
['U', 'D', 'U', 'U', 'U', 'D']
[430.97, 431.8, 431.52, 430.18, 428.78] [429.85, 429.48, 428.5, 427.2]

DN(U)


2016-01-06 22:00:00
['U', 'U', 'D', 'D', 'U', 'D']
[430.98, 430.96, 430.19, 430.99, 430.97] [430.26, 429.97, 429.85]



2016-01-07 01:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[430.82, 430.5, 430.52, 430.98, 430.96] [430.15, 430.26, 429.97]
DN(U)

UP(D)

UP(D)



DN(D)


UP(U)
2016-01-07 13:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[446.94, 446.04, 428.64] [443.36, 428.35, 426.12]
UP(D)
UP(U)
UP(U)
UP(D)
2016-01-07 17:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[455.24, 455.27, 446.01] [450.27, 445.15, 442.49]
UP(D)
UP(D)
2016-01-07